In [1]:
import keras

Using TensorFlow backend.


In [2]:
print('keras: ', keras.__version__)

keras:  2.2.0


In [3]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [4]:
# Get images
image_raw = img_to_array(load_img('dog.jpg'))
image_raw = np.array(image_raw, dtype=float)
#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)

In [5]:
X = rgb2lab(1.0/255*image_raw)[:,:,0]
Y = rgb2lab(1.0/255*image_raw)[:,:,1:]
Y /= 128
X = X.reshape(1, 256, 256, 1)
Y = Y.reshape(1, 256, 256, 2)

In [6]:
def conv_stack(data, filters, s):
        output = Conv2D(filters, (3, 3), strides=s, activation='relu', padding='same')(data)
        output = BatchNormalization()(output)
        return output

#Create inception embedding
img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(299, 299))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
embed = inception.predict(img)
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = conv_stack(encoder_input, 64, 2)
encoder_output = conv_stack(encoder_output, 128, 2)
encoder_output = conv_stack(encoder_output, 256, 2)
encoder_output = conv_stack(encoder_output, 512, 1)
encoder_output = conv_stack(encoder_output, 256, 1)

#Fusion
# y_mid: (None, 256, 28, 28)
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Permute((2, 1))(fusion_output) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([fusion_output, encoder_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu')(fusion_output) 

#Decoder
decoder_output = UpSampling2D((2, 2))(fusion_output)
decoder_output = conv_stack(decoder_output, 128, 1)
decoder_output = conv_stack(decoder_output, 64, 1)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = conv_stack(decoder_output, 32, 1)
decoder_output = conv_stack(decoder_output, 16, 1)
decoder_output = Conv2D(2, (2, 2), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [7]:
# Finish model
model.compile(optimizer='rmsprop', loss='mse')
model.fit([X, embed], Y,
    batch_size=1,
    epochs=100)

Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 0.4254
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.2114
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 0.1157
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.0736
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.0519
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.0381
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.0299
Epoch 8/100
1/1 [==============================] - 1s 1s/step - loss: 0.0277
Epoch 9/100
1/1 [==============================] - 1s 1s/step - loss: 0.0247
Epoch 10/100
1/1 [==============================] - 1s 1s/step - loss: 0.0250
Epoch 11/100
1/1 [==============================] - 1s 1s/step - loss: 0.0238
Epoch 12/100
1/1 [==============================] - 1s 1s/step - loss: 0.0187
Epoch 13/100
1/1 [==============================] - 1s 1s/step - loss: 0.

In [8]:
output = model.predict([X, embed])
output *= 128
# Output colorizations
cur = np.zeros((256, 256, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [ ]:
#embed_output = UpSampling2D((2, 2))(embed_input)
#embed_output = UpSampling2D((2, 2))(embed_input)
# embed_output= Conv2D(512, (3, 3), activation="relu", strides=2)(embed_input)
# embed_output= Conv2D(512, (3, 3), activation="relu", strides=2)(embed_output)
# embed_output = Dense(1024, activation='relu')(embed_output)
# embed_output = Dense(512, activation='relu')(embed_output)
# embed_output = Dense(256, activation='relu')(embed_output)
# embed_output = Reshape(([256,]))(embed_output)